In [ ]:
!pip install python-binance pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.4 MB/s eta 0:00:00


In [ ]:
from binance.client import Client
import pandas as pd
import numpy as np
from datetime import datetime

client = Client()

def to_milliseconds(date_str):
    dt = datetime.strptime(date_str, "%Y-%m-%d")
    return int(dt.timestamp() * 1000)

def get_klines(symbol='WIFUSDT', interval='1h', start_time=None, end_time=None, limit=1000):
    df_total = pd.DataFrame()
    while True:
        klines = client.get_klines(symbol=symbol, interval=interval, startTime=start_time, endTime=end_time, limit=limit)
        if not klines:
            break
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'number_of_trades',
            'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore'
        ])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df = df[['close', 'volume']].astype(float)
        df_total = pd.concat([df_total, df])
        if len(klines) < limit:
            break
        start_time = int(klines[-1][0]) + 1
    return df_total

def add_ema_crossover(df, fast, slow, label):
    ema_fast = df['close'].ewm(span=fast, adjust=False).mean()
    ema_slow = df['close'].ewm(span=slow, adjust=False).mean()
    df[label] = (ema_fast > ema_slow).astype(int)
    return df



# Time windows
start_15m = to_milliseconds("2024-10-01")
midpoint_15m = to_milliseconds("2025-03-01")
end_15m = to_milliseconds("2025-06-28")

# Fetch and build all dataframes
df_15m = pd.concat([
    get_klines('WIFUSDT', '15m', start_time=start_15m, end_time=midpoint_15m),
    get_klines('WIFUSDT', '15m', start_time=midpoint_15m, end_time=end_15m)
])
df_15m = add_ema_crossover(df_15m, 9, 21, '15M_9C21')
df_15m = add_ema_crossover(df_15m, 21, 50, '15M_21C50')

df_1h = pd.concat([
    get_klines('WIFUSDT', '1h', start_time=start_15m, end_time=midpoint_15m),
    get_klines('WIFUSDT', '1h', start_time=midpoint_15m, end_time=end_15m)
])
df_1h = add_ema_crossover(df_1h, 12, 26, '1H_12C26')
df_1h = add_ema_crossover(df_1h, 13, 34, '1H_13C34')

df_4h = pd.concat([
    get_klines('WIFUSDT', '4h', start_time=start_15m, end_time=midpoint_15m),
    get_klines('WIFUSDT', '4h', start_time=midpoint_15m, end_time=end_15m)
])
df_4h = add_ema_crossover(df_4h, 20, 50, '4H_20C50')
df_4h = add_ema_crossover(df_4h, 50, 200, '4H_50C200')

# View result
print(df_15m[['close', '15M_9C21', '15M_21C50']].tail())
print(df_1h[['close', '1H_12C26', '1H_13C34']].tail())
print(df_4h[['close', '4H_20C50', '4H_50C200']].tail())


BinanceAPIException: APIError(code=0): Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error.

In [ ]:
def add_volume_features(df, label_prefix):
    df[f'{label_prefix}_vol_ema20'] = df['volume'].ewm(span=20).mean()
    df[f'{label_prefix}_volume_norm'] = df['volume'] / df[f'{label_prefix}_vol_ema20']
    df[f'{label_prefix}_volume_spike'] = (df['volume'] > 1.5 * df[f'{label_prefix}_vol_ema20']).astype(int)
    return df
# For 15m
df_15m = add_volume_features(df_15m, '15M')
df_1h = add_volume_features(df_1h, '1H')
df_4h = add_volume_features(df_4h, '4H')

In [ ]:
# prompt: read SSL_T.xlsx file

file_path = 'SSL_T.xlsx'
try:
    df_excel = pd.read_excel(file_path)
    print(df_excel.head()) # Print the first few rows to verify
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")


   S/l Start Date Start Time Criteria  Candle/Days             Signal  Entry  \
0    1 2024-10-23   09:15:00   LG PDL          1.0        LW Doji 78%  2.360   
1    2 2024-10-23   13:00:00   LG RGC          2.0         BBR Candle  2.418   
2    3 2024-10-23   15:45:00   LG PDH         13.0  Green Doji LW 61%  2.394   
3    4 2024-10-23   21:15:00   LG PDL          5.0       BBG + BBG LW  2.534   
4    5 2024-10-23   22:00:00   LG RGC          2.0     Bearish Hammer  2.495   

   Stop Loss  Profit Target Result  ...        Fee  Remaining Balance     R  \
0      2.352          2.389      W  ...  15.127500       10451.872500  5.00   
1      2.411          2.443      W  ...  15.089250       10555.783250  7.00   
2      2.376          2.444      W  ...  15.186960       10789.876290  6.56   
3      2.614          2.509      L  ...   7.538258       10732.338032  4.00   
4      2.486          2.550      L  ...  14.973000       10681.365032  6.11   

   Target     CR  C Target  Comment    Cum L

In [ ]:
df_excel.head()

,S/l,Start Date,Start Time,Criteria,Candle/Days,Signal,Entry,Stop Loss,Profit Target,Result,...,Fee,Remaining Balance,R,Target,CR,C Target,Comment,Cum Loss,Exit Date,Exit Time
0,1,2024-10-23,09:15:00,LG PDL,1.0,LW Doji 78%,2.360,2.352,2.389,W,...,15.127500,10451.872500,5.00,NaN,11.00,0.78,100%/DL,0.000000,2024-10-23,09:30:00
1,2,2024-10-23,13:00:00,LG RGC,2.0,BBR Candle,2.418,2.411,2.443,W,...,15.089250,10555.783250,7.00,NaN,7.00,PMiddleL,L50,0.000000,2024-10-23,13:45:00
2,3,2024-10-23,15:45:00,LG PDH,13.0,Green Doji LW 61%,2.394,2.376,2.444,W,...,15.186960,10789.876290,6.56,NaN,4.83,PmiddleL,PO/DL,0.000000,2024-10-23,17:30:00
3,4,2024-10-23,21:15:00,LG PDL,5.0,BBG + BBG LW,2.534,2.614,2.509,L,...,7.538258,10732.338032,4.00,NaN,NaN,PO,LGD,57.538258,2024-10-23,21:45:00
4,5,2024-10-23,22:00:00,LG RGC,2.0,Bearish Hammer,2.495,2.486,2.550,L,...,14.973000,10681.365032,6.11,NaN,NaN,0.5,DL,108.511258,2024-10-23,22:30:00


In [ ]:
import pandas as pd

# Convert Start Date + Start Time to a single datetime
df_excel['datetime'] = pd.to_datetime(df_excel['Start Date'].astype(str) + ' ' + df_excel['Start Time'].astype(str))

# Create proper 15m and 1h floors
df_excel['datetime_15m'] = df_excel['datetime'].dt.floor('15min')
df_excel['datetime_1h'] = df_excel['datetime'].dt.floor('H')

# Custom floor for 4H (to match Binance's actual 4h candle opens)
def binance_4h_floor(dt):
    floored_hour = (dt.hour // 4) * 4
    return pd.Timestamp(dt.year, dt.month, dt.day, floored_hour)

df_excel['datetime_4h'] = df_excel['datetime'].apply(binance_4h_floor)

# Ensure index of EMA dataframes are also properly floored
df_15m.index = pd.to_datetime(df_15m.index).floor('15min')
df_1h.index = pd.to_datetime(df_1h.index).floor('H')
df_4h.index = df_4h.index.map(binance_4h_floor)  # apply custom 4H floor

# Merge all EMA data
df_excel = df_excel.merge(
    df_15m[['15M_9C21', '15M_21C50', '15M_volume_spike', '15M_volume_norm']],
    how='left',
    left_on='datetime_15m',
    right_index=True
)

df_excel = df_excel.merge(
    df_1h[['1H_12C26', '1H_13C34', '1H_volume_spike', '1H_volume_norm']],
    how='left',
    left_on='datetime_1h',
    right_index=True
)

df_excel = df_excel.merge(
    df_4h[['4H_20C50', '4H_50C200', '4H_volume_spike', '4H_volume_norm']],
    how='left',
    left_on='datetime_4h',
    right_index=True
)

# Drop temp datetime columns (optional)


/tmp/ipython-input-39-2114267937.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_excel['datetime_1h'] = df_excel['datetime'].dt.floor('H')
/tmp/ipython-input-39-2114267937.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_1h.index = pd.to_datetime(df_1h.index).floor('H')


In [ ]:
df_excel.head()

,S/l,Start Date,Start Time,Criteria,Candle/Days,Signal,Entry,Stop Loss,Profit Target,Result,...,15M_volume_spike,15M_volume_norm,1H_12C26_y,1H_13C34_y,1H_volume_spike,1H_volume_norm,4H_20C50_y,4H_50C200_y,4H_volume_spike,4H_volume_norm
0,1,2024-10-23,09:15:00,LG PDL,1.0,LW Doji 78%,2.360,2.352,2.389,W,...,0,0.818026,0,0,1,1.830014,0,1,0,1.313976
1,2,2024-10-23,13:00:00,LG RGC,2.0,BBR Candle,2.418,2.411,2.443,W,...,0,0.423907,0,0,0,1.173363,0,1,0,1.466423
2,3,2024-10-23,15:45:00,LG PDH,13.0,Green Doji LW 61%,2.394,2.376,2.444,W,...,0,0.925175,0,0,1,1.664125,0,1,0,1.466423
3,4,2024-10-23,21:15:00,LG PDL,5.0,BBG + BBG LW,2.534,2.614,2.509,L,...,0,0.725476,1,1,0,1.082022,0,1,0,1.336720
4,5,2024-10-23,22:00:00,LG RGC,2.0,Bearish Hammer,2.495,2.486,2.550,L,...,0,0.161006,1,1,0,0.417406,0,1,0,1.336720


In [ ]:
df_excel.drop(columns=['datetime', 'datetime_15m', 'datetime_1h', 'datetime_4h'], inplace=True)

In [ ]:
# prompt: delete duplicate value from df_excel based on S/l column

df_excel.drop_duplicates(subset=['S/l'], inplace=True, keep='first')

print(f"\nNumber of rows after dropping duplicates: {len(df_excel)}")
df_excel.head(1)

In [ ]:
# prompt: save df_excel

df_excel.to_excel('SSL_T_output.xlsx', index=False)